In [2]:
import ast
import json

from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import os
import wandb

In [3]:
api = wandb.Api()

In [4]:
def get_experiment_data(filters, name):
    runs = api.runs("bartekcupial/sf2_nethack", filters=filters)
    data = []
    for run in runs:
        try:
            artifact = api.artifact(f"bartekcupial/sf2_nethack/run-{run.id}-table_results:v1")
            artifact_path = Path("artifacts") / f"run-{run.id}-table_results:v1" / "table_results.table.json"
            if artifact_path.exists():
                table = wandb.Table.from_json(json.load(open(artifact_path, "r+")), artifact)
            else:
                table = artifact.get("table_results")
            df = table.get_dataframe()
            df["seed"] = run.config["seed"]
            df["train/env_steps"] = run.summary_metrics["train/env_steps"]
            df["train_dir"] = run.config["train_dir"]
            df["name"] = name
            df["exp_tag"] = ast.literal_eval(run.config["exp_tags"])[0]
            data.append(df)
        except Exception as e:
            pass

    df = pd.concat(data, axis=0).reset_index()
    df.reset_index(drop=True, inplace=True)
    return df

In [5]:
data = get_experiment_data({"config.exp_tags": "['2024_02_02_eval_full']"}, "all")

In [13]:
data["exp_tag"] = data["train_dir"].apply(lambda x: "-".join(x.split("/")[-2].split("-")[:-1]))

In [14]:
dd = pd.read_csv("notebooks/cloned_expert_500_human_monk.csv")
dd["exp_tag"] = "pretrained"
data = pd.concat([data, dd], axis=0)

In [15]:
data[["score", "exp_tag"]].groupby("exp_tag").count()

,score
exp_tag,
2024-01-23-monk-appo,500
2024-01-23-monk-appo-bc-t,389
2024-01-23-monk-appo-ks-t,499
2024-01-23-monk-appo-t,515
2024-01-29-monk-appo-ewc-t,488
pretrained,482


0       True
1       True
2       True
3       True
4       True
       ...  
477     True
478     True
479     True
480    False
481     True
Name: death, Length: 2873, dtype: bool

In [20]:
data[data["death"] == "quit"][["score", "exp_tag"]].groupby("exp_tag").mean()

,score
exp_tag,
2024-01-23-monk-appo,653.295699
2024-01-23-monk-appo-bc-t,7568.108696
2024-01-23-monk-appo-ks-t,10296.000000
2024-01-23-monk-appo-t,498.069182
2024-01-29-monk-appo-ewc-t,3030.245763
pretrained,3492.406015


In [19]:
data[data["death"] != "quit"][["score", "exp_tag"]].groupby("exp_tag").mean()

,score
exp_tag,
2024-01-23-monk-appo,875.458599
2024-01-23-monk-appo-bc-t,7202.364431
2024-01-23-monk-appo-ks-t,10582.916840
2024-01-23-monk-appo-t,1230.939086
2024-01-29-monk-appo-ewc-t,4096.883784
pretrained,5215.005731


In [ ]:
for tag in data["exp_tag"].unique():
    method_data = data[data["exp_tag"] == tag]
    method_data.loc[method_data["death"].str.contains("lack of food|starvation"), "death"] = "lack of food|starvation"
    unique, counts = np.unique(method_data["death"].to_numpy(), return_counts=True)

    c = counts[counts.argsort()[::-1]] / len(method_data) * 100
    u = unique[counts.argsort()[::-1]]
    df = pd.DataFrame({"chance": c, "death": u})
    
    print(f"Top 10 most common deaths for: {tag}")
    print(df[:10].to_string(float_format="{:0.2f}%".format, index=False))

Top 10 most common deaths for: 2024-01-23-monk-appo-ks-t
 chance                                  death
 38.48%                lack of food|starvation
  3.61%                killed by a soldier ant
  3.61%                                   quit
  2.61%              killed by a white unicorn
  2.00% poisoned by a rotted glob of gray ooze
  2.00%   killed by the wrath of Chih Sung-tzu
  1.80%             petrified by a chickatrice
  1.80%               killed by a Woodland-elf
  1.40%                       killed by a wand
  1.40%                      killed by a guard
Top 10 most common deaths for: 2024-01-23-monk-appo-bc-t
 chance                                    death
 50.13%                  lack of food|starvation
 11.83%                                     quit
  2.06%     killed by the wrath of Chih Sung-tzu
  1.80%                     killed by a hill orc
  1.29%                         killed by a wand
  1.29%                  killed by a soldier ant
  1.03%                 ki